In [106]:
import pandas as pd
import numpy as np
import math

In [216]:
def calculate_competitveness_factor(row, factor=2): 
    if (math.isnan(row['TCO_VAR']) == False): 
        A = row['TCO_AVG'] - row['TCO_MIN']
        B = row['TCO_AVG_POWERTRAIN'] - row['TCO_MIN']
        F = row['TCO_AVG'] / factor
        
        return np.exp(0.5 * (A / F)**2) * np.exp(-0.5 * (B / F)**2)
    else: 
        return np.nan
    
def calculate_veh_cot_factor(row, factor=1.5): 
    try:
        if (math.isnan(row['veh_price_ihs_avg_previous']) == False): 
            A = row['veh_price_ihs_avg_previous'] - row['veh_price_km_min']
            B = row['veh_price_km'] - row['veh_price_km_min']
            F = row['veh_price_ihs_avg_previous'] / factor

            return np.exp(0.5 * (A / F)**2) * np.exp(-0.5 * (B / F)**2)
        else: 
            return np.nan
    except: 
        return np.nan

def define_powertrain(row):
    if ((row['e_propulsion_system_design'] == 'ICE') | (row['e_propulsion_system_design'] == 'ICE: Stop/Start')) &\
    ((row['e_fuel_type'] == 'DIESEL') | (row['e_fuel_type'] == 'DIESEL-CNG')):
        return "ICE-D"
    
    if ((row['e_propulsion_system_design'] == 'ICE') | (row['e_propulsion_system_design'] == 'ICE: Stop/Start')) &\
        ((row['e_fuel_type'] == 'GAS') | (row['e_fuel_type'] == 'GAS-CNG') |\
         (row['e_fuel_type'] == 'GAS-E100') | (row['e_fuel_type'] == 'GAS-E85') | \
         (row['e_fuel_type'] == 'GAS-LPG') | (row['e_fuel_type'] == 'GAS-M100')):
        return "ICE-G"

    if ((row['e_propulsion_system_design'] == 'ICE') | (row['e_propulsion_system_design'] == 'ICE: Stop/Start')) &\
        (row['e_fuel_type'] == 'CNG'):
        return "CNG"

    if ((row['e_propulsion_system_design'] == 'ICE') | (row['e_propulsion_system_design'] == 'ICE: Stop/Start')) &\
        (row['e_fuel_type'] == 'Hydrogen'):
        return "HICEV"

    if ((row['e_propulsion_system_design'] == 'ICE') | (row['e_propulsion_system_design'] == 'ICE: Stop/Start')) &\
        (row['e_fuel_type'] == 'LPG'):
        return "ICE-LPG"

    if  ((row['e_propulsion_system_design'] == "Hybrid-Mild") |\
        ( (row['e_propulsion_system_design'] == "Hybrid-Full") &\
         (row['e_propulsion_system_design2'] in ('HEV-Full (DIESEL)', 'HEV-Full (GAS)', 'HEV-Full (GAS-E100)',
                                                  'HEV-Full (GAS-E85)', 'HEV-Full (GAS-M100)', 'HEV-Full (LPG)')))):
        return "HEV"

    if (row['e_propulsion_system_design'] == "Hybrid-Full") &\
        ((row['e_propulsion_system_design2'] in ("PHEV-Full (DIESEL)" , "PHEV-Full (GAS)" , "PHEV-Full (GAS-E100)"))):
        return "PHEV"
    if (row['e_propulsion_system_design'] == "Electric"):
        return "BEV"

    if (row['e_propulsion_system_design'] == "Electric") &\
         ((row['e_fuel_type'] in ('GAS', 'GAS-E100'))):
        return "REEV"

    if (row['e_propulsion_system_design'] == "Fuel Cell"):
        return "FCEV"

    
def s3_correction_market_share(row):
    try: 
        if ((row['ratio_volume_previous'] + row["s2_ms_normalize_var"]) < 0): 
            return 0

        elif ((row['ratio_volume_previous'] + row["s2_ms_normalize_var"]) > 1):
            return 1-row['ratio_volume_previous'] 
        else: 
            return row["s2_ms_normalize_var"]
    except: 
        return np.nan

def s2_correction_market_share(row):
    try: 
        if ((row['ratio_volume_previous'] + row["s1_ms_normalize_var"]) < 0): 
            return -row['ratio_volume_previous'] / 2

        elif ((row['ratio_volume_previous'] + row["s1_ms_normalize_var"]) > 1):
            return 1-row['ratio_volume_previous'] 
        else: 
            return row["s1_ms_normalize_var"]
    except: 
        return np.nan
    
def s1_correction_market_share(row):
    try: 
        if ((row['ratio_volume_previous'] + row["s1_variation_factor"]) < 0): 
            return -row['ratio_volume_previous'] 

        elif ((row['ratio_volume_previous'] + row["s1_variation_factor"]) > 1):
            return 1-row['ratio_volume_previous'] 
        else: 
            return row["s1_variation_factor"]
    except: 
        return np.nan

In [242]:
prop_powertrain = pd.read_csv("/home/cdsw/data/powertrain_propulsion_system.csv")

prop_powertrain.rename(columns={"propulsion_system_design": "e_propulsion_system_design2"}, inplace=True)
prop_powertrain['e_propulsion_system_design2'] = prop_powertrain['e_propulsion_system_design2'].str.lower()

ihs = pd.read_csv("/home/cdsw/data/ihs-202302.csv")


volumes = ihs\
    .groupby(['region', 'period_year', 'e_propulsion_system_design', 'e_propulsion_system_design2', 'e_fuel_type'])\
    .agg({"sum_volume" : sum}).reset_index()

volumes['powertrain'] = volumes.apply(lambda row: define_powertrain(row), axis=1)


agg_volume_powertrain = volumes.groupby(["region","period_year","powertrain"])\
    .agg({"sum_volume": sum})\
    .transform({"sum_volume": lambda x: x / 1E6})

sum_agg = agg_volume_powertrain.reset_index().groupby(['region','period_year']).sum().rename(columns={"sum_volume": "sum_powertrain"})

agg_volume_powertrain_per = pd.merge(agg_volume_powertrain.reset_index(), sum_agg, on=["region","period_year"])\
.assign(ratio_volume= lambda x: x["sum_volume"] / x["sum_powertrain"])\
.rename(columns={"period_year": "year", "powertrain": "powertrain_type"})\
.assign(region = lambda x: x['region'].str.lower())

In [281]:
agg_volume_powertrain_per.loc[(agg_volume_powertrain_per.year == 2022) & (agg_volume_powertrain_per.region == 'western europe'), :]

,region,year,powertrain_type,sum_volume,sum_powertrain,ratio_volume
1233,western europe,2022,BEV,1.203415,9.934707,0.121132
1234,western europe,2022,FCEV,0.000184,9.934707,0.000019
1235,western europe,2022,HEV,2.039382,9.934707,0.205279
1236,western europe,2022,ICE-D,2.369351,9.934707,0.238492
1237,western europe,2022,ICE-G,3.537112,9.934707,0.356036
1238,western europe,2022,PHEV,0.785263,9.934707,0.079042


In [313]:
veh_cost = pd.read_csv('data/output/veh_cost_output.csv')
tco_data = pd.read_csv('data/output/TCO.csv')


tco_data = pd.merge(tco_data, 
                    veh_cost[['powertrain_type', 'region', 'year', 'veh_price']],
                    on = ['powertrain_type', 'region', 'year'])



tco_data = pd.merge(tco_data, 
                    agg_volume_powertrain_per[['powertrain_type', 'region', 'year', 'ratio_volume']],
                    on = ['powertrain_type', 'region', 'year'], how="left")

tco_data['ratio_volume'] = tco_data['ratio_volume'].fillna(0)

tco_data['TCO_AVG_POWERTRAIN'] = tco_data.groupby(['region', 'year','powertrain_type'])["TCO_percent"].transform(sum)
tco_data['TCO_AVG'] = tco_data.groupby(['region', 'year'])["TCO_AVG_POWERTRAIN"].transform(np.mean)
tco_data['TCO_MIN'] = tco_data.groupby(['region', 'year'])["TCO_AVG_POWERTRAIN"].transform(min)



tco_data['veh_price_km'] = tco_data['veh_price'] / 1000
tco_data['veh_price_km_avg'] = tco_data.groupby(['region', 'year'])["veh_price_km"].transform(np.mean)
tco_data['veh_price_km_min'] = tco_data.groupby(['region', 'year'])["veh_price_km"].transform(min)

tco_data = tco_data[['powertrain_type', 'region', 'year', 'TCO_AVG_POWERTRAIN', 'TCO_AVG','TCO_MIN', 'veh_price_km','veh_price_km_avg', 'ratio_volume', 'veh_price_km_min']].drop_duplicates()
tco_data['TCO_VAR'] = tco_data.groupby(['powertrain_type', 'region'])['TCO_AVG_POWERTRAIN'].pct_change() 

tco_data['veh_price_ihs']= tco_data['veh_price_km'] * tco_data['ratio_volume']
tco_data['veh_price_ihs_avg'] = tco_data.groupby(['region', 'year'])['veh_price_ihs'].transform(sum)
s = tco_data[['region', 'year', 'veh_price_ihs_avg']].drop_duplicates()
s['veh_price_ihs_avg_previous'] = s['veh_price_ihs_avg'].shift(1)
tco_data = pd.merge(tco_data, s.drop(['veh_price_ihs_avg'], axis=1), on = ['region', 'year'])

tco_data['TCO_Compet_factor'] = tco_data.apply(lambda x: calculate_competitveness_factor(x, factor = 2), axis=1)
tco_data['veh_cost_factor'] = tco_data.apply(lambda x: calculate_veh_cot_factor(x, factor = 1.5), axis=1)

s = tco_data[['region', 'year', 'powertrain_type', 'ratio_volume']].drop_duplicates()
s['ratio_volume_previous'] = s.groupby(['region','powertrain_type'])['ratio_volume'].shift(1)
s['ratio_volume_previous_minus'] = s.groupby(['region','powertrain_type'])['ratio_volume'].shift(2)
tco_data = pd.merge(tco_data, s.drop(['ratio_volume'], axis=1), on = ['region', 'year', 'powertrain_type'])
tco_data['s1_variation_factor'] = ((- tco_data['veh_cost_factor'] * tco_data['TCO_Compet_factor'] * tco_data['TCO_VAR']) \
                                   + ((tco_data['TCO_Compet_factor'] -1) * tco_data['veh_cost_factor']))\
                                    * (1 - tco_data['ratio_volume_previous'])

tco_data['s1_market_share_correction'] = tco_data.apply(lambda row : s1_correction_market_share(row), axis=1)

tco_data["s1_ms_normalize_var"] = tco_data['s1_market_share_correction'] -\
(tco_data["ratio_volume_previous_minus"] * tco_data.groupby(['region', 'year'])['s1_market_share_correction'].transform(sum))

tco_data['s2_market_share_correction'] = tco_data.apply(lambda row : s2_correction_market_share(row), axis=1)
tco_data["s2_ms_normalize_var"] = tco_data['s2_market_share_correction'] -\
(tco_data["ratio_volume_previous"] * tco_data.groupby(['region', 'year'])['s2_market_share_correction'].transform(sum))

tco_data['s3_market_share_correction'] = tco_data.apply(lambda row : s3_correction_market_share(row), axis=1)
tco_data["s3_ms_normalize_var"] = tco_data['s3_market_share_correction'] -\
(tco_data["ratio_volume_previous"] * tco_data.groupby(['region', 'year'])['s3_market_share_correction'].transform(sum))

tco_data['ms_normalize_var_cumsum'] = tco_data.groupby(['region','powertrain_type'])['s3_ms_normalize_var'].transform(np.cumsum)

In [314]:
tco_data

,powertrain_type,region,year,TCO_AVG_POWERTRAIN,TCO_AVG,TCO_MIN,veh_price_km,veh_price_km_avg,ratio_volume,veh_price_km_min,...,ratio_volume_previous,ratio_volume_previous_minus,s1_variation_factor,s1_market_share_correction,s1_ms_normalize_var,s2_market_share_correction,s2_ms_normalize_var,s3_market_share_correction,s3_ms_normalize_var,ms_normalize_var_cumsum
0,ICE-G,western europe,2021,1.503917,1.577098,1.438995,26.800000,32.532429,0.385425,26.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ICE-D,western europe,2021,1.592239,1.577098,1.438995,28.800000,32.532429,0.288441,26.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CNG,western europe,2021,1.526376,1.577098,1.438995,28.717000,32.532429,0.000000,26.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HEV,western europe,2021,1.438995,1.577098,1.438995,28.740000,32.532429,0.168145,26.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PHEV,western europe,2021,1.459306,1.577098,1.438995,31.560000,32.532429,0.076144,26.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,CNG,western europe,2040,1.536864,1.589331,1.419819,28.717000,32.142732,0.000000,28.717,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,HEV,western europe,2040,1.512622,1.589331,1.419819,29.817620,32.142732,0.000000,28.717,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,PHEV,western europe,2040,1.602481,1.589331,1.419819,34.470480,32.142732,0.000000,28.717,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,BEV,western europe,2040,1.419819,1.589331,1.419819,34.984550,32.142732,0.000000,28.717,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [315]:
tco_data\
.query("region == 'western europe' and year < 2030 and powertrain_type == 'ICE-G'")\
.iloc[:15, :].transpose()

,0,7,14,21,28,35,42,49,56
powertrain_type,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G
region,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe
year,2021,2022,2023,2024,2025,2026,2027,2028,2029
TCO_AVG_POWERTRAIN,1.50392,1.50466,1.50541,1.50617,1.73818,1.73903,1.73989,1.74075,1.74163
TCO_AVG,1.5771,1.56254,1.54898,1.53634,1.60101,1.59009,1.57993,1.60776,1.59897
TCO_MIN,1.439,1.43883,1.43871,1.43862,1.43291,1.41709,1.40194,1.50792,1.49478
veh_price_km,26.8,26.8,26.8,26.8,30.99,30.99,30.99,30.99,30.99
veh_price_km_avg,32.5324,32.176,31.8426,31.5308,32.6646,32.3914,32.1355,32.7887,32.5642
ratio_volume,0.385425,0.356036,0.276148,0.174331,0.107558,0.043454,0.0202969,0.0106687,0.00656606
veh_price_km_min,26.8,26.8,26.8,26.8,28.717,28.717,28.717,28.717,28.717


In [316]:
vol = tco_data[tco_data['year'] == 2023][['powertrain_type', 'year', 'region', 'ratio_volume']].rename(columns={'ratio_volume': 'base_volume'})

tco_data_preview = pd.merge(tco_data, vol, on = ['powertrain_type', 'region'])\
.query("year_x > year_y").drop(['year_y'], axis = 1)\
.rename(columns={'year_x': 'year'})


tco_data_preview["actual_volume"] = tco_data_preview["base_volume"] + tco_data_preview["s3_ms_normalize_var"]

In [317]:
tco_data_preview.transpose()

,3,4,5,6,7,8,9,10,11,12,...,130,131,132,133,134,135,136,137,138,139
powertrain_type,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,ICE-G,...,FCEV,FCEV,FCEV,FCEV,FCEV,FCEV,FCEV,FCEV,FCEV,FCEV
region,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe,...,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe,western europe
year,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
TCO_AVG_POWERTRAIN,1.50617,1.73818,1.73903,1.73989,1.74075,1.74163,1.74251,1.74341,1.74431,1.74522,...,1.61729,1.60127,1.58576,1.57073,1.55618,1.54209,1.52845,1.51524,1.50357,1.50354
TCO_AVG,1.53634,1.60101,1.59009,1.57993,1.60776,1.59897,1.59381,1.61336,1.60954,1.60588,...,1.61336,1.60954,1.60588,1.60239,1.59905,1.59587,1.59284,1.58995,1.5886,1.58933
TCO_MIN,1.43862,1.43291,1.41709,1.40194,1.50792,1.49478,1.46992,1.49614,1.48393,1.47224,...,1.49614,1.48393,1.47224,1.46105,1.45035,1.44011,1.43031,1.42094,1.41918,1.41982
veh_price_km,26.8,30.99,30.99,30.99,30.99,30.99,30.99,30.99,30.99,30.99,...,36.6921,36.3187,35.9566,35.6056,35.2651,34.935,34.6148,34.3043,34.0295,34.0295
veh_price_km_avg,31.5308,32.6646,32.3914,32.1355,32.7887,32.5642,32.4253,32.8709,32.7632,32.6593,...,32.8709,32.7632,32.6593,32.5591,32.4624,32.3691,32.2791,32.1922,32.1427,32.1427
ratio_volume,0.174331,0.107558,0.043454,0.0202969,0.0106687,0.00656606,0.00281,0,0,0,...,0,0,0,0,0,0,0,0,0,0
veh_price_km_min,26.8,28.717,28.717,28.717,28.717,28.717,28.717,28.717,28.717,28.717,...,28.717,28.717,28.717,28.717,28.717,28.717,28.717,28.717,28.717,28.717
